In [2]:
import pandas as pd
import numpy as np
from scipy import special

import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve

# Motivational Example A - Logistic MTA with No Sequence

This notebook will build a basic logistic model on MTA data with no sequence. 

It will model on frequency of touchpoints in a journey. 

It will arbitrarily enforce a non data driven 45 day window (perhaps provided by our marketing SME). You are encouraged to also look into data driven ways to define the proper window or model directly on touchpoint proximity.

In [5]:
sequence_df = pd.read_csv('../datasets/sequence_fact.csv')
sequence_df.head(10)

sequence_id        fullVisitorId      event_name       event_datetime  \
0  0099Rqojoj1MCXN  7343617347507729080  organic_search  2018-04-15 17:31:50   
1  0099Rqojoj1MCXN  7343617347507729080        dead_end  2018-04-15 17:33:05   
2  00A9Lkka73okUx2    89656057821147903  organic_search  2017-09-14 16:36:56   
3  00A9Lkka73okUx2    89656057821147903        dead_end  2017-09-14 16:54:09   
4  00B30tmbMwJn7Cf  4307745811624101170  organic_search  2017-04-21 02:41:23   
5  00B30tmbMwJn7Cf  4307745811624101170        dead_end  2017-04-21 02:41:24   
6  00BKxKnEYlKbw9b  7129167701457127936  organic_search  2016-10-02 15:16:09   
7  00BKxKnEYlKbw9b  7129167701457127936        dead_end  2016-10-02 15:16:10   
8  00EttOfsTTyp45B  3217678225016118393        referral  2017-10-23 19:44:20   
9  00EttOfsTTyp45B  3217678225016118393        dead_end  2017-10-23 19:46:43   

   conversion_proximity  
0                  75.0  
1                   0.0  
2                1033.0  
3                   0.0  
4                   1.0  
5                   0.0  
6                   1.0  
7                   0.0  
8                 143.0  
9                   0.0

In [7]:
sequence_to_visitor_map = sequence_df[['sequence_id','fullVisitorId']].drop_duplicates().reset_index(drop=True)

## Make a modeling dataset

We want to filter out rows where conversion proximity >= 45 days (45 days *  86400 seconds per day
 = 3,888,000 seconds).

We want 1 row to represent the sequence id.

For fun, lets build a string column that shows the events in order

Next there is a conversion column 1 = yes conversion 0 equals dead end journey

Finally, since this a non sequence table we would need to make a column based on each channel if it is present in the 45 day journey. 

In [8]:
## filter conversion_proximity 
model_prep_df1 = sequence_df.loc[(sequence_df['conversion_proximity']/86400)<=45,:]

In [9]:
## make the sequence details
model_prep_df2 = model_prep_df1.groupby('sequence_id')['event_name'].agg(lambda x: '>'.join(x)).reset_index()
model_prep_df2.columns = ['sequence_id','sequence_details']
model_prep_df2.head()

sequence_id         sequence_details
0  0099Rqojoj1MCXN  organic_search>dead_end
1  00A9Lkka73okUx2  organic_search>dead_end
2  00B30tmbMwJn7Cf  organic_search>dead_end
3  00BKxKnEYlKbw9b  organic_search>dead_end
4  00EttOfsTTyp45B        referral>dead_end

In [10]:
## make the modeling features
model_prep_df3 = model_prep_df1.pivot_table(index='sequence_id', columns='event_name', aggfunc='size', fill_value=0).reset_index()
model_prep_df3 = model_prep_df3.rename_axis(None, axis=1)
model_prep_df3.head()

sequence_id  (other)  affiliates  conversion  dead_end  direct  \
0  0099Rqojoj1MCXN        0           0           0         1       0   
1  00A9Lkka73okUx2        0           0           0         1       0   
2  00B30tmbMwJn7Cf        0           0           0         1       0   
3  00BKxKnEYlKbw9b        0           0           0         1       0   
4  00EttOfsTTyp45B        0           0           0         1       0   

   display  organic_search  paid_search  referral  social  
0        0               1            0         0       0  
1        0               1            0         0       0  
2        0               1            0         0       0  
3        0               1            0         0       0  
4        0               0            0         1       0

In [11]:
## Final joining and prep
model_prep_df4 = model_prep_df2.merge(model_prep_df3, on='sequence_id',how='left')

## Add visitor id back in
model_prep_df4 = model_prep_df4.merge(sequence_to_visitor_map, on='sequence_id',how='left')


## drop dead_end and move Y to last spot
model_data_final =  model_prep_df4[['fullVisitorId','sequence_id','sequence_details','affiliates'
                                 ,'direct','display','organic_search'
                                 ,'paid_search','referral','social'
                                 ,'(other)','conversion']]
model_data_final.head()

fullVisitorId      sequence_id         sequence_details  affiliates  \
0  7343617347507729080  0099Rqojoj1MCXN  organic_search>dead_end           0   
1    89656057821147903  00A9Lkka73okUx2  organic_search>dead_end           0   
2  4307745811624101170  00B30tmbMwJn7Cf  organic_search>dead_end           0   
3  7129167701457127936  00BKxKnEYlKbw9b  organic_search>dead_end           0   
4  3217678225016118393  00EttOfsTTyp45B        referral>dead_end           0   

   direct  display  organic_search  paid_search  referral  social  (other)  \
0       0        0               1            0         0       0        0   
1       0        0               1            0         0       0        0   
2       0        0               1            0         0       0        0   
3       0        0               1            0         0       0        0   
4       0        0               0            0         1       0        0   

   conversion  
0           0  
1           0  
2           0  
3           0  
4           0

In [14]:
model_data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99718 entries, 0 to 99717
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   fullVisitorId     99718 non-null  object
 1   sequence_id       99718 non-null  object
 2   sequence_details  99718 non-null  object
 3   affiliates        99718 non-null  int64 
 4   direct            99718 non-null  int64 
 5   display           99718 non-null  int64 
 6   organic_search    99718 non-null  int64 
 7   paid_search       99718 non-null  int64 
 8   referral          99718 non-null  int64 
 9   social            99718 non-null  int64 
 10  (other)           99718 non-null  int64 
 11  conversion        99718 non-null  int64 
dtypes: int64(9), object(3)
memory usage: 9.9+ MB


In [9]:
vis_detail = pd.read_csv('../datasets/visitor_detail.csv')
# vis_detail.head(10)

We will split the data into 60 20 20 train validate test 

- train is used to fit models. Validate is used while making model selection feature selection decisions. Test is a true holdout for final reporting to stakeholders.

In [16]:
# Split the data into train, validation, and test sets
temp_data, test_data = train_test_split(model_prep_df4, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(temp_data, test_size=0.25, random_state=42)

In [17]:
ind_vars = ['affiliates','direct','display','organic_search','paid_search','referral','social','(other)']
dep_var = 'conversion'

# Separate the features and target variable
X_train = train_data[ind_vars]
y_train = train_data[dep_var]
X_val = val_data[ind_vars]
y_val = val_data[dep_var]
X_test = test_data[ind_vars]
y_test = test_data[dep_var]

In [52]:
# Define the parameter grid for grid search
param_grid = {
    'C': [0.1, 1, 10, 100]
}

lr_model = LogisticRegression()

grid_search = GridSearchCV(lr_model, param_grid, cv=5, scoring='neg_log_loss')
grid_search.fit(X_train, y_train)

lr_model = LogisticRegression(**grid_search.best_params_)
lr_model.fit(X_train, y_train)

LogisticRegression(C=100)

In [53]:
train_pred = lr_model.predict_proba(X_train)[:, 1]
val_pred = lr_model.predict_proba(X_val)[:, 1]

train_auc = roc_auc_score(y_train, train_pred)
test_auc = roc_auc_score(y_val, val_pred)

print("Train AUC:", train_auc)
print("Test AUC:", test_auc)

Train AUC: 0.8036488658056495
Test AUC: 0.8066017391060295


In [63]:
fig = go.Figure()
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

for i, j in enumerate(['train','validate']):
    
    
    if j == 'train':
        fpr, tpr, _ = roc_curve(y_train, train_pred)
        auc_score = roc_auc_score(y_train, train_pred)
    elif j == 'validate':
        fpr, tpr, _ = roc_curve(y_val, val_pred)
        auc_score = roc_auc_score(y_val, val_pred)

    name = "{} AUC= {:.2f}".format(j.title(),auc_score)
    fig.add_trace(go.Scatter(x=fpr, y=tpr, name=name, mode='lines'))

fig.update_layout(
    title='AUC for Train and Validate'
    ,xaxis_title='False Positive Rate'
    ,yaxis_title='True Positive Rate'
    ,yaxis=dict(scaleanchor="x", scaleratio=1)
    ,xaxis=dict(constrain='domain')
    ,width=700
    ,height=500
)
fig.show()

In [76]:
# Get the coefficients (weights) for each variable
coefficients = lr_model.coef_

print("logisitc coefficients in log-odds")
print("")
# Print the coefficients
for i, feature_name in enumerate(X_train.columns):

    print("{}: {:.2f}".format(feature_name,coefficients[0][i]))

logisitc coefficients in log-odds

affiliates: -9.54
direct: 0.12
display: -0.07
organic_search: 0.16
paid_search: 0.55
referral: 0.84
social: -2.69
(other): -0.04


In [86]:
# Get the coefficients (weights) for each variable
coefficients = lr_model.coef_

print("logisitc coefficients")
print("")
# Print the coefficients
for i, feature_name in enumerate(X_train.columns):
    odds_ratio = np.exp(coefficients[0][i])
    probability_impact = np.exp(coefficients[0][i]) - 1
    print("{} | odds ratio: {:.2f}, probability impact: {:.2f}".format(feature_name,odds_ratio,probability_impact))

logisitc coefficients

affiliates | odds ratio: 0.00, probability impact: -1.00
direct | odds ratio: 1.13, probability impact: 0.13
display | odds ratio: 0.93, probability impact: -0.07
organic_search | odds ratio: 1.17, probability impact: 0.17
paid_search | odds ratio: 1.73, probability impact: 0.73
referral | odds ratio: 2.31, probability impact: 1.31
social | odds ratio: 0.07, probability impact: -0.93
(other) | odds ratio: 0.96, probability impact: -0.04


To interpret this: a 1 unit increase in direct channel touchpoints results in the odds of conversion increasing by 13%.

## Deploying these weights into a business system/ attribution report dataset

So now how can we logically create a data driven logisitc regression input that would let us divy up conversions appropriately?

To divy up these credits we would need to meet the following rules.
- If a touchpoint is present but no conversion it gets zero credit for a conversion
- If a touchpoint is present but there is a conversion it gets >= 0 credit for a conversion
- If a touchpoint not present it gets no credit and it gets no penalty either.

The softmax function transforms the original logistic coefficients into a set of values that sum up to 1, with each value representing the probability weight of the corresponding independent variable in contributing to the outcome variable. These probability weights can be interpreted as the relative importance of each independent variable in predicting the outcome variable.

In [83]:
## example interpretation

coefficients = special.softmax(lr_model.coef_)
print("logisitc coefficients in softmax")
print("")
# Print the coefficients
for i, feature_name in enumerate(X_train.columns):

    print("{}: {:.5f}".format(feature_name,coefficients[0][i]))

logisitc coefficients in softmax

affiliates: 0.00001
direct: 0.13573
display: 0.11199
organic_search: 0.14151
paid_search: 0.20818
referral: 0.27844
social: 0.00819
(other): 0.11595


These probability weights can be interpreted as follows: referral has the highest probability weight of 0.278, which means it is the most important independent variable in predicting the outcome variable. 

Assume we had 1 touchpoint of each channel present and then a conversion. These set of numbers could represent how much credit each channel gets in that scenario. 

Now what if we saw a journey with 2 paid search 1 referal 1 affiliate then a conversion

The conversion should be divvyd up like this

In [95]:
channel_impacts = 2 * (0.20818) + 1 * (0.27844) + 1 * (0.00001)

paid_search_channel_contribution = (2 * (0.20818)) / channel_impacts
paid_search_touchpoint_contribution = paid_search_channel_contribution / 2

referral_channel_contribution = (1 * (0.27844)) / channel_impacts
referral_touchpoint_contribution = referral_channel_contribution / 1

affiliate_channel_contribution = (1 * (0.00001)) / channel_impacts
affiliate_touchpoint_contribution = affiliate_channel_contribution / 1

print("paid search channel credit: {:.3f}".format(paid_search_channel_contribution))
print("paid search per touchpoint credit: {:.3f}".format(paid_search_touchpoint_contribution))
print("")
print("referral channel credit: {:.3f}".format(referral_channel_contribution))
print("referral touchpoint credit: {:.3f}".format(referral_touchpoint_contribution))
print("")
print("affiliate channel credit: {:.3f}".format(affiliate_channel_contribution))
print("affiliate touchpoint credit: {:.3f}".format(affiliate_touchpoint_contribution))

paid search channel credit: 0.599
paid search per touchpoint credit: 0.300

referral channel credit: 0.401
referral touchpoint credit: 0.401

affiliate channel credit: 0.000
affiliate touchpoint credit: 0.000


In summary, we can use the softmax transformation on the logistic regression coefficients to give us indepentent variable impacts on a scale between 0 and 1 and summing up to 1

These weights are data driven and sensible to then be deployed to calculate channel contribution and touchpoint contribution like the example above. 

Once the entire dataset is scored, we can now slice and dice the results. 

Let's talk about some limitations of this approach in its currently status now.

We only get 1 weight/coefficient for a given channel touchpoint that doesn't inherently change given the presence of other touchpoints so the interaction effects are limitied just to the attribution games' nature of the canabalism. Other approaches may actually let these base weights change given control variables and synergies across the channels. 

Weights could change based on
1) synergyistic effects
2) special patterns or sequences
3) control variables (geography, customer demographics, etc.)

## What Next

**Try similar approach with these better algorithms (does not address the mission of sequences)**
- Check for multi-colliniatry try logistic Bagging
- Try xgboost

**Try markov chain**
- you will need to research what type of input data is needed

**Build position based only (non-sequence) solutions**
- number of features = last N positions * 8 channels
- eg affiliate|1, affiliate|2, affiliate|3 ... etc

**Build small-sequence solutions with log regression, log regression bagged, xgboost**
- number of features permutations of 1 touchpoint present (8 channels) + permutations of 2 tactic + permutations of 3 tactic

**Build large scale sequence solutions with embedding, dimensionality reduction, sequence mining techniques**
- Figuring out hot to embbed or train machine learning algoirthms in the highly complex and spare sequences data

**go straight for LSTM**
- you will need to research what type of input data is needed